In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import geopandas as gpd
import seaborn as sns
# from shapely.geometry import Point, Polygon
import statistics
import folium



In [2]:
data= pd.read_csv("./novel-corona-virus-2019-dataset/covid_19_data.csv")
data.head()
data.info()

# Convert Last Update column to datetime64 format

data['Date'] = data['ObservationDate'].apply(pd.to_datetime)
data.drop(['SNo'],axis=1,inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13209 entries, 0 to 13208
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   SNo              13209 non-null  int64  
 1   ObservationDate  13209 non-null  object 
 2   Province/State   6827 non-null   object 
 3   Country/Region   13209 non-null  object 
 4   Last Update      13209 non-null  object 
 5   Confirmed        13209 non-null  float64
 6   Deaths           13209 non-null  float64
 7   Recovered        13209 non-null  float64
dtypes: float64(3), int64(1), object(4)
memory usage: 619.2+ KB


In [3]:
data.head()

,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Date
0,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0,2020-01-22
1,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0,2020-01-22
2,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0,2020-01-22
3,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0,2020-01-22
4,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0,2020-01-22


In [4]:
china_data =data[data['Country/Region'].isin(['Mainland China', 'Hong Kong', 'Taiwan', 'Macau'])]
china_data.head()


,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Date
0,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0,2020-01-22
1,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0,2020-01-22
2,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0,2020-01-22
3,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0,2020-01-22
4,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0,2020-01-22


In [5]:
other_data = data[~data['Country/Region'].isin(['Mainland China', 'Hong Kong', 'Taiwan', 'Macau'])]
other_data.head()

,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Date
31,01/22/2020,Washington,US,1/22/2020 17:00,1.0,0.0,0.0,2020-01-22
35,01/22/2020,NaN,Japan,1/22/2020 17:00,2.0,0.0,0.0,2020-01-22
36,01/22/2020,NaN,Thailand,1/22/2020 17:00,2.0,0.0,0.0,2020-01-22
37,01/22/2020,NaN,South Korea,1/22/2020 17:00,1.0,0.0,0.0,2020-01-22
69,01/23/2020,Washington,US,1/23/20 17:00,1.0,0.0,0.0,2020-01-23


In [6]:
d = data['Date'][-1:].astype('str')
year = int(d.values[0].split('-')[0])
month = int(d.values[0].split('-')[1])
day = int(d.values[0].split('-')[2].split()[0])

from datetime import date
data_latest = data[data['Date'] >= pd.Timestamp(date(year,month,day))]


In [7]:
china_latest = data_latest[data_latest['Country/Region'].isin(['Mainland China', 'Hong Kong', 'Taiwan', 'Macau'])]

In [8]:
other_latest = data_latest[~data_latest['Country/Region'].isin(['Mainland China', 'Hong Kong', 'Taiwan', 'Macau'])]

In [9]:
countries_regions = other_data['Country/Region'].unique().tolist() + chinaData['Country/Region'].unique().tolist()
print(countries_regions)
print(len(countries_regions))

NameError: name 'chinaData' is not defined

In [ ]:
# Creating a dataframe with total no of confirmed cases for every country
Number_of_countries = len(data_latest['Country/Region'].value_counts())
print(Number_of_countries)

cases = pd.DataFrame(data_latest.groupby('Country/Region')['Confirmed'].sum())
cases['Country/Region'] = cases.index
cases.index=np.arange(1,Number_of_countries+1)

global_cases = cases[['Country/Region','Confirmed']]
#global_cases.sort_values(by=['Confirmed'],ascending=False)
global_cases



In [ ]:
world_coordinates = pd.read_csv('./world-coordinates/world_coordinates.csv')
world_coordinates.rename(columns={'Country':'Country/Region'},inplace = True)
world_coordinates.head()
# Merging the coordinates dataframe with original dataframe
world_data = pd.merge(world_coordinates,global_cases,on='Country/Region')
world_data.head()

In [ ]:
# Creating a dataframe with total no of confirmed cases for every country
Number_of_countries = len(data_latest['Country/Region'].value_counts())

cases = pd.DataFrame(data_latest.groupby('Country/Region')['Confirmed'].sum())
cases['Country/Region'] = cases.index
cases.index=np.arange(1,Number_of_countries+1)

global_cases = cases[['Country/Region','Confirmed']]
#global_cases.sort_values(by=['Confirmed'],ascending=False)
global_cases.head()

In [ ]:
import folium
# create map and display it
world_map = folium.Map(location=[10, -20], zoom_start=2.3,tiles='Stamen Toner')

for lat, lon, value, name in zip(world_data['latitude'], world_data['longitude'], world_data['Confirmed'], world_data['Country/Region']):
    folium.CircleMarker([lat, lon],
                        radius=10,
                        popup = ('<strong>Country</strong>: ' + str(name).capitalize() + '<br>'
                                '<strong>Confirmed Cases</strong>: ' + str(value) + '<br>'),
                        color='red',
                        
                        fill_color='red',
                        fill_opacity=0.7 ).add_to(world_map)
world_map

In [ ]:

china_coordinates = pd.read_csv('./china-coordinates/china_coordinates.csv')
china_coordinates.head()

In [ ]:
data_latest.head()

In [ ]:
# chinaData
# china_coordinates.rename(columns={'Province':'Province/State'},inplace = True)
# china_coordinates.head()
# china_data_latest = chinaData 



# d = data['Date'][-1:].astype('str')
# year = int(d.values[0].split('-')[0])
# month = int(d.values[0].split('-')[1])
# day = int(d.values[0].split('-')[2].split()[0])

# from datetime import date

# china_data_latest = chinaData[chinaData['Date'] >= pd.Timestamp(date(year,month,day))]
# china_data_latest[]


In [ ]:

chinaData['ObservationDate'] = chinaData['ObservationDate'].apply(pd.to_datetime)


d = chinaData['ObservationDate'][-1:].astype('str')
year = int(d.values[0].split('-')[0])
month = int(d.values[0].split('-')[1])
day = int(d.values[0].split('-')[2].split()[0])


from datetime import date
china_data_latest = chinaData[data['ObservationDate'] >= pd.Timestamp(date(year,month,day))]
china_data_latest.head()